In [1]:
""" Description of table 
Will get the conversion rates, density, mean synaptic volume for all the presyn, postsyn pairs

"""

' Description of table \nWill get the conversion rates, density, mean synaptic volume for all the presyn, postsyn pairs\n\n'

In [2]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


# AUTOMATICALLY GENERATING CODE FOR CONTACT PRE POST

In [ ]:
compartment_type = {
"apical":2,
"basal":3,
"oblique":4,
"soma":5,
"axon":7,
"dendrites":8
}
#ones not doing:   "total":, "axon":


# total_length ="sum(postsyn_length * (majority_label=2))",
# n_syn ="sum( n_synapses * (majority_label=2))",
# n_contacts="sum(majority_label=2)",
# contact_conversion="sum( n_synapses * (majority_label=2))/sum(majority_label=2)",
# contact_density="sum( n_synapses * (majority_label=2))/sum( postsyn_length * (majority_label=2))"

# data_type = {
#     "total_length":["bigint unsigned","total postsynaptic contact length","sum( postsyn_length * (majority_label=","))"],
#     "n_syn":["bigint unsigned","number of synapses","sum( n_synapses * (majority_label=","))"],
#     "n_contacts":["bigint unsigned","number of contacts","sum(majority_label=",")"],
#     "contact_conversion":["float","synapse to contact ratio","sum( n_synapses * (majority_label=","))"
#                           ,"/sum(majority_label=",")"],
#     "contact_density":["float","synapse to contact length ratio","sum( n_synapses * (majority_label=","))/",
#                       "sum( postsyn_length * (majority_label=","))"]
# }

data_type = {
"n_contacts":["bigint unsigned","number of contacts","sum(majority_label=",")"],
"postsyn_length":["bigint unsigned","total postsynaptic contact length","sum( postsyn_length * (majority_label=","))"],
"contact_conversion":["float","synapse to contact ratio","if(sum(majority_label=",")=0,null,","sum( n_synapses * (majority_label=","))"
                  ,"/sum(majority_label=","))"],
"contact_density":["float","synapse to contact length ratio","if(sum( postsyn_length * (majority_label=","))=0,null,"
               ,"sum( n_synapses * (majority_label=","))/",
              "sum( postsyn_length * (majority_label=",")))"],
"n_synapses":["bigint unsigned","total number of synapses for neurite",
         "sum(n_synapses * (majority_label=","))"],
"synapse_sizes_mean":["float","average synaptic size","if((n_synapses * (majority_label=","))=0,0,","sum((n_synapses * (majority_label=","))","*synapse_sizes_mean*(majority_label=","))",
                                  "/sum(n_synapses * (majority_label=",")))"]
}



# synapses = sum(n_synapses)
# contacts = count(*)
# contact_conversion = sum(n_synapses)/count(*)
# contact_density = sum(n_synapses)/sum(post_syn length)
    
    #genereate the table header

# extra_headers = ["total"]
# for c in list(compartment_type.keys()) + extra_headers:
#     for keys,values in data_type.items():
#         print(str(c) + "_" + str(keys) + "   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        
# print("\n\n")

extra_headers = ["total"]
for c in extra_headers + list(compartment_type.keys()) :
    for keys,values in data_type.items():
        if values[0] != "float":
            print(str(c) + "_" + str(keys) + "   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        else:
            print(str(c) + "_" + str(keys) + "=null   :" + str(values[0]) + " #" + str(c) + " " + str(values[1]))
        
print("\n\n")


for keys_comp,values_comp in compartment_type.items():
    print("")
    for keys_data,values_data in data_type.items():
        base_string = str(keys_comp) + "_" + str(keys_data) + "= " 
        
        start_counter = 2
        rounds = (len(values_data)-2)/2
        base_string += '"'
        for i in range(0,int(rounds)):
            if keys_comp != "axon":
                base_string += values_data[start_counter] + str(values_comp) + values_data[start_counter + 1]
            else:
                base_string += values_data[start_counter] + "6 OR majority_label=7" + values_data[start_counter + 1]
            
            start_counter +=2
        base_string += '",'
        print(base_string)

# MAKING PRE POST TEST TABLE

In [6]:
#ta3p100.ContactPrePost2.drop()

In [7]:
labels_to_keep = [2,3,4,5,6,7,8]

@schema
class ContactPrePost2(dj.Computed):
    definition="""
    -> ta3p100.Segment
    postsyn :bigint unsigned #id of the postsynaptic neuron
    ---
    total_n_contacts   :bigint unsigned #total number of contacts
    total_postsyn_length   :bigint unsigned #total total postsynaptic contact length
    total_contact_conversion=null   :float #total synapse to contact ratio
    total_contact_density=null   :float #total synapse to contact length ratio
    total_n_synapses   :bigint unsigned #total total number of synapses for neurite
    total_synapse_sizes_mean=null   :float #total average synaptic size
    apical_n_contacts   :bigint unsigned #apical number of contacts
    apical_postsyn_length   :bigint unsigned #apical total postsynaptic contact length
    apical_contact_conversion=null   :float #apical synapse to contact ratio
    apical_contact_density=null   :float #apical synapse to contact length ratio
    apical_n_synapses   :bigint unsigned #apical total number of synapses for neurite
    apical_synapse_sizes_mean=null   :float #apical average synaptic size
    basal_n_contacts   :bigint unsigned #basal number of contacts
    basal_postsyn_length   :bigint unsigned #basal total postsynaptic contact length
    basal_contact_conversion=null   :float #basal synapse to contact ratio
    basal_contact_density=null   :float #basal synapse to contact length ratio
    basal_n_synapses   :bigint unsigned #basal total number of synapses for neurite
    basal_synapse_sizes_mean=null   :float #basal average synaptic size
    oblique_n_contacts   :bigint unsigned #oblique number of contacts
    oblique_postsyn_length   :bigint unsigned #oblique total postsynaptic contact length
    oblique_contact_conversion=null   :float #oblique synapse to contact ratio
    oblique_contact_density=null   :float #oblique synapse to contact length ratio
    oblique_n_synapses   :bigint unsigned #oblique total number of synapses for neurite
    oblique_synapse_sizes_mean=null   :float #oblique average synaptic size
    soma_n_contacts   :bigint unsigned #soma number of contacts
    soma_postsyn_length   :bigint unsigned #soma total postsynaptic contact length
    soma_contact_conversion=null   :float #soma synapse to contact ratio
    soma_contact_density=null   :float #soma synapse to contact length ratio
    soma_n_synapses   :bigint unsigned #soma total number of synapses for neurite
    soma_synapse_sizes_mean=null   :float #soma average synaptic size
    axon_n_contacts   :bigint unsigned #axon number of contacts
    axon_postsyn_length   :bigint unsigned #axon total postsynaptic contact length
    axon_contact_conversion=null   :float #axon synapse to contact ratio
    axon_contact_density=null   :float #axon synapse to contact length ratio
    axon_n_synapses   :bigint unsigned #axon total number of synapses for neurite
    axon_synapse_sizes_mean=null   :float #axon average synaptic size
    dendrites_n_contacts   :bigint unsigned #dendrites number of contacts
    dendrites_postsyn_length   :bigint unsigned #dendrites total postsynaptic contact length
    dendrites_contact_conversion=null   :float #dendrites synapse to contact ratio
    dendrites_contact_density=null   :float #dendrites synapse to contact length ratio
    dendrites_n_synapses   :bigint unsigned #dendrites total number of synapses for neurite
    dendrites_synapse_sizes_mean=null   :float #dendrites average synaptic size
    """
    
    key_source = ta3p100.Segmentation & ta3p100.CurrentSegmentation
    
    
    def make(self,key):

        contact_pre_post = dj.U("segmentation","segment_id","postsyn").aggr(
            ta3p100.Contact2.proj("postsyn","postsyn_length","n_synapses","majority_label","synapse_sizes_mean",segment_id="presyn")
            & [dict(majority_label=x) for x in labels_to_keep],

            
            total_n_contacts= "count(*)",
            total_postsyn_length= "sum(postsyn_length)",
            total_contact_conversion= "sum( n_synapses )/count(*)",
            total_contact_density= "sum( n_synapses )/sum(postsyn_length)",
            total_n_synapses= "sum(n_synapses)",
            #total_synapse_sizes_mean= "sum(n_synapses*synapse_sizes_mean)/sum(n_synapses)",
            total_synapse_sizes_mean= "if(sum(n_synapses)=0,0,sum(n_synapses*synapse_sizes_mean)/sum(n_synapses))",


            apical_n_contacts= "sum(majority_label=2)",
            apical_postsyn_length= "sum( postsyn_length * (majority_label=2))",
            apical_contact_conversion= "if(sum(majority_label=2)=0,null,sum( n_synapses * (majority_label=2))/sum(majority_label=2))",
            apical_contact_density= "if(sum( postsyn_length * (majority_label=2))=0,null,sum( n_synapses * (majority_label=2))/sum( postsyn_length * (majority_label=2)))",
            apical_n_synapses= "sum(n_synapses * (majority_label=2))",
            apical_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=2))=0,0,sum((n_synapses * (majority_label=2))*synapse_sizes_mean*(majority_label=2))/sum(n_synapses * (majority_label=2)))",

            basal_n_contacts= "sum(majority_label=3)",
            basal_postsyn_length= "sum( postsyn_length * (majority_label=3))",
            basal_contact_conversion= "if(sum(majority_label=3)=0,null,sum( n_synapses * (majority_label=3))/sum(majority_label=3))",
            basal_contact_density= "if(sum( postsyn_length * (majority_label=3))=0,null,sum( n_synapses * (majority_label=3))/sum( postsyn_length * (majority_label=3)))",
            basal_n_synapses= "sum(n_synapses * (majority_label=3))",
            basal_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=3))=0,0,sum((n_synapses * (majority_label=3))*synapse_sizes_mean*(majority_label=3))/sum(n_synapses * (majority_label=3)))",

            oblique_n_contacts= "sum(majority_label=4)",
            oblique_postsyn_length= "sum( postsyn_length * (majority_label=4))",
            oblique_contact_conversion= "if(sum(majority_label=4)=0,null,sum( n_synapses * (majority_label=4))/sum(majority_label=4))",
            oblique_contact_density= "if(sum( postsyn_length * (majority_label=4))=0,null,sum( n_synapses * (majority_label=4))/sum( postsyn_length * (majority_label=4)))",
            oblique_n_synapses= "sum(n_synapses * (majority_label=4))",
            oblique_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=4))=0,0,sum((n_synapses * (majority_label=4))*synapse_sizes_mean*(majority_label=4))/sum(n_synapses * (majority_label=4)))",

            soma_n_contacts= "sum(majority_label=5)",
            soma_postsyn_length= "sum( postsyn_length * (majority_label=5))",
            soma_contact_conversion= "if(sum(majority_label=5)=0,null,sum( n_synapses * (majority_label=5))/sum(majority_label=5))",
            soma_contact_density= "if(sum( postsyn_length * (majority_label=5))=0,null,sum( n_synapses * (majority_label=5))/sum( postsyn_length * (majority_label=5)))",
            soma_n_synapses= "sum(n_synapses * (majority_label=5))",
            soma_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=5))=0,0,sum((n_synapses * (majority_label=5))*synapse_sizes_mean*(majority_label=5))/sum(n_synapses * (majority_label=5)))",

            axon_n_contacts= "sum(majority_label=6 OR majority_label=7)",
            axon_postsyn_length= "sum( postsyn_length * (majority_label=6 OR majority_label=7))",
            axon_contact_conversion= "if(sum(majority_label=6 OR majority_label=7)=0,null,sum( n_synapses * (majority_label=6 OR majority_label=7))/sum(majority_label=6 OR majority_label=7))",
            axon_contact_density= "if(sum( postsyn_length * (majority_label=6 OR majority_label=7))=0,null,sum( n_synapses * (majority_label=6 OR majority_label=7))/sum( postsyn_length * (majority_label=6 OR majority_label=7)))",
            axon_n_synapses= "sum(n_synapses * (majority_label=6 OR majority_label=7))",
            axon_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=6 OR majority_label=7))=0,0,sum((n_synapses * (majority_label=6 OR majority_label=7))*synapse_sizes_mean*(majority_label=6 OR majority_label=7))/sum(n_synapses * (majority_label=6 OR majority_label=7)))",

            dendrites_n_contacts= "sum(majority_label=8)",
            dendrites_postsyn_length= "sum( postsyn_length * (majority_label=8))",
            dendrites_contact_conversion= "if(sum(majority_label=8)=0,null,sum( n_synapses * (majority_label=8))/sum(majority_label=8))",
            dendrites_contact_density= "if(sum( postsyn_length * (majority_label=8))=0,null,sum( n_synapses * (majority_label=8))/sum( postsyn_length * (majority_label=8)))",
            dendrites_n_synapses= "sum(n_synapses * (majority_label=8))",
            dendrites_synapse_sizes_mean= "if(sum(n_synapses * (majority_label=8))=0,0,sum((n_synapses * (majority_label=8))*synapse_sizes_mean*(majority_label=8))/sum(n_synapses * (majority_label=8)))",
        )
        
        
        self.insert(contact_pre_post,skip_duplicates=True)

In [8]:
start_time = time.time()
ContactPrePost2.populate()
print(f"Total time: {time.time()-start_time}")

Total time: 402.68739461898804


In [ ]:
ContactPrePost2()

In [ ]:
ContactPrePost()

In [ ]:
FinalPrePostTest()

In [4]:
ta3p100.Contact2()

segmentation segmentation id,contact_id,presyn segment id unique within each Segmentation,postsyn segment id unique within each Segmentation,contact_x,contact_y,contact_z,presyn_length,postsyn_length,n_synapses,synapse_sizes_mean,majority_label,majority_label_ratio
2,4542184,648518346341351467,648518346341357297,210603,244565,66108,7164.08,9167.32,0,0.0,8,1.0
2,4542185,648518346341351467,648518346341357297,213313,253183,62096,12258.6,11296.3,1,192.0,8,1.0
2,4542186,648518346341351467,648518346341366977,215801,256906,72309,9788.65,14899.5,1,1889.0,8,1.0
2,4542187,648518346341351467,648518346341366977,221684,263963,73118,3900.87,3345.71,0,0.0,8,1.0
2,4542188,648518346341351467,648518346341373284,215417,256048,58533,8017.33,9948.91,1,192.0,8,1.0
2,4542189,648518346341351467,648518346341373284,221795,263574,73588,7781.78,8825.41,0,0.0,8,1.0
2,4542190,648518346341351467,648518346341373284,224940,266938,76924,3310.29,3290.89,0,0.0,8,1.0
2,4542191,648518346341351467,648518346341379609,222144,203456,63533,7926.69,7251.01,0,0.0,8,1.0
2,4542192,648518346341351467,648518346341379951,208486,217722,60357,5503.03,5645.07,0,0.0,8,1.0
2,4542193,648518346341351467,648518346341379951,216449,254785,70779,28255.5,83132.0,1,1889.0,8,1.0
